In [1]:
import numpy as np
import os
import sys
import torch
import pytorch3d

import matplotlib.pyplot as plt

import trimesh
from pytorch3d.io import load_objs_as_meshes, save_obj
import numpy as np
from tqdm.notebook import tqdm



from pytorch3d.loss import (
    chamfer_distance, 
    mesh_edge_loss, 
    mesh_laplacian_smoothing, 
    mesh_normal_consistency,
)
from pytorch3d.io import load_objs_as_meshes, save_obj

from pytorch3d.loss import (
    chamfer_distance, 
    mesh_edge_loss, 
    mesh_laplacian_smoothing, 
    mesh_normal_consistency,
)

# Data structures and functions for rendering
from pytorch3d.structures import Meshes
from pytorch3d.renderer import (
    look_at_view_transform,
    OpenGLPerspectiveCameras, 
    PointLights, 
    DirectionalLights, 
    Materials, 
    RasterizationSettings, 
    MeshRenderer,
    MeshRasterizer,
    SoftPhongShader,
    SoftSilhouetteShader,
    SoftPhongShader,
    TexturesVertex,
)

from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.loss import (
    chamfer_distance, 
    mesh_edge_loss, 
    mesh_laplacian_smoothing, 
    mesh_normal_consistency,
)

from src.plot_image_grid import image_grid
from src.cleansed_cube import Geoid

In [2]:
import torch
import torch.nn as nn




In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")


#obj_filename = "./data/bunny.obj"
#obj_filename = "./data/nefertiti.obj"
obj_filename = "./data/centered_3.obj"
bunny = load_objs_as_meshes([obj_filename], device=device)

textures=TexturesVertex(verts_features=[torch.ones_like(bunny.verts_packed())])

bunny = Meshes(verts=[bunny.verts_packed()], 
           faces=[bunny.faces_packed()],
           textures=textures)

verts = bunny.verts_packed()
N = verts.shape[0]
center = verts.mean(0)
scale = max((verts - center).abs().max(0)[0])
bunny.offset_verts_(-center)
bunny.scale_verts_((1.0 / float(scale)));

In [15]:
n = 32

geoid = Geoid(n).to(device)
#rs = torch.randn(3, requires_grad=True, device=device)

#optimizer = torch.optim.SGD([rs], lr=1., momentum=0.)
optimizer = torch.optim.Adam(geoid.parameters(), lr=0.001)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

In [16]:
Niter = 10001
num_views_per_iteration = 2

loop = tqdm(range(Niter))

laplace_weight =  1.

w_chamfer = 1.0
w_laplacian = 0.1 
w_normal = 0.01 
w_edge = 1.0 

for i in loop:
    stop = False
    
    optimizer.zero_grad()
        
    verts, faces, colors = geoid()

    textures = TexturesVertex(verts_features=[torch.ones_like(verts)])
    p3d_mesh = Meshes(verts=[verts],
                  faces=[faces],
                  textures=textures)
    
    sample_src = sample_points_from_meshes(p3d_mesh, 10000)
    sample_trg = sample_points_from_meshes(bunny, 10000)
    
    loss_chamfer, _ = chamfer_distance(sample_trg, sample_src)
    loss_laplacian = mesh_laplacian_smoothing(p3d_mesh, method="uniform")
    loss_normal = mesh_normal_consistency(p3d_mesh)
    loss_edge = mesh_edge_loss(p3d_mesh)
    
    
    
    
    #laplacian_smoothing = mesh_laplacian_smoothing(new_src_mesh, method="uniform")
    sum_loss = torch.tensor(0.0, device=device) 
    
    sum_loss += loss_chamfer * w_chamfer
    #sum_loss += laplace_loss * w_laplacian
    sum_loss += loss_laplacian * w_laplacian
    sum_loss += loss_normal * w_normal
    sum_loss += loss_edge * w_edge
    
    loop.set_description("total_loss = %.6f" % sum_loss)
    
    sum_loss.backward()
    #cube.smooth()
    optimizer.step()
    
    if i % 500 == 0:
        f = f'./data/geoid_mesh_{n}_{i}.obj'
        geoid.export(f)
        print(f)
#kernel = 0
f = f'./data/geoid_mesh_{n}.obj'
geoid.export(f)
f

  0%|          | 0/10001 [00:00<?, ?it/s]

./data/geoid_mesh_32_0.obj
./data/geoid_mesh_32_500.obj
./data/geoid_mesh_32_1000.obj
./data/geoid_mesh_32_1500.obj
./data/geoid_mesh_32_2000.obj
./data/geoid_mesh_32_2500.obj
./data/geoid_mesh_32_3000.obj
./data/geoid_mesh_32_3500.obj
./data/geoid_mesh_32_4000.obj
./data/geoid_mesh_32_4500.obj
./data/geoid_mesh_32_5000.obj
./data/geoid_mesh_32_5500.obj
./data/geoid_mesh_32_6000.obj
./data/geoid_mesh_32_6500.obj
./data/geoid_mesh_32_7000.obj
./data/geoid_mesh_32_7500.obj
./data/geoid_mesh_32_8000.obj
./data/geoid_mesh_32_8500.obj
./data/geoid_mesh_32_9000.obj
./data/geoid_mesh_32_9500.obj
./data/geoid_mesh_32_10000.obj


'./data/geoid_mesh_32.obj'